## Combining the MF catalgoues
### Creates the file "name_field"_all.fits, for all fields
### The RA and Dec from "name_field"_all.fits is used as input for XID+

In [63]:
import numpy as np
from astropy.io import fits
from astropy.table import Table, vstack
import os
%matplotlib inline  

import matplotlib.pyplot as plt
from astropy.wcs import WCS
from matplotlib.patches import Ellipse
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Column


In [71]:
# The SPIRE fields in HELP:
all_names =  ['GAMA-09_SPIRE','GAMA-12_SPIRE','GAMA-15_SPIRE','HATLAS-NGP_SPIRE','HATLAS-SGP_SPIRE','SSDF_SPIRE','AKARI-SEP_SPIRE','Bootes_SPIRE','CDFS-SWIRE_SPIRE','COSMOS_SPIRE','EGS_SPIRE',\
              'ELAIS-N1_SPIRE','ELAIS-N2_SPIRE','ELAIS-S1_SPIRE','HDF-N_SPIRE','Lockman-SWIRE_SPIRE','SA13_SPIRE',\
              'SPIRE-NEP_SPIRE','xFLS_SPIRE','XMM-13hr_SPIRE','XMM-LSS_SPIRE']


In [39]:
band = ['250','350','500']

In [40]:
# loops over the matched filterd catalogues and creates a merged catalgoue for every field.
# Duplicated are removed in the 350um catalogue when the object is within a 12 arcsec radius of a 250um detection
# Duplicated are removed in the 500um catalogue when the object is within a 18 arcsec radius of a 250um or
# 350um detection

for j in range(np.size(all_names)):
    name = all_names[j]+'250_cat_MF.fits'
    cat1=Table.read(name)

    RA,DEC = cat1['RA'],cat1['Dec']
    cat1.add_column(Column(np.zeros(np.size(RA))),name='flag')

    cat1['F_BLIND_pix_SPIRE_250'].name = 'F_BLIND_pix_SPIRE'
    cat1['FErr_BLIND_pix_SPIRE_250'].name = 'FErr_BLIND_pix_SPIRE'

    name = all_names[j]+'350_cat_MF.fits'
    cat2=Table.read(name)

    RA_m,DEC_m = cat2['RA'],cat2['Dec']
    cat2.add_column(Column(np.zeros(np.size(RA_m))+1),name='flag')

    cat2['F_BLIND_pix_SPIRE_350'].name = 'F_BLIND_pix_SPIRE'
    cat2['FErr_BLIND_pix_SPIRE_350'].name = 'FErr_BLIND_pix_SPIRE'


    catalog = SkyCoord(ra=RA, dec=DEC)  
    c = SkyCoord(ra=RA_m, dec=DEC_m)

    idx, d2d, d3d = c.match_to_catalog_sky(catalog) 
    use = np.array(d2d)*3600 > 12.
    cat2 = cat2[use]

    cat_all = vstack([cat1, cat2])

    name = all_names[j]+'500_cat_MF.fits'
    cat3=Table.read(name)

    RA_m,DEC_m = cat3['RA'],cat3['Dec']
    cat3.add_column(Column(np.zeros(np.size(RA_m))+2),name='flag')

    cat3['F_BLIND_pix_SPIRE_500'].name = 'F_BLIND_pix_SPIRE'
    cat3['FErr_BLIND_pix_SPIRE_500'].name = 'FErr_BLIND_pix_SPIRE'

    catalog = SkyCoord(ra=cat_all['RA'], dec=cat_all['Dec'])  
    c = SkyCoord(ra=RA_m, dec=DEC_m)

    idx, d2d, d3d = c.match_to_catalog_sky(catalog) 
    use = np.array(d2d)*3600 > 18.
    cat3 = cat3[use]

    cat_all = vstack([cat_all, cat3])

    cat_all.write(all_names[j]+'_all.fits', format='fits',overwrite=True)

